In [1]:
from functools import reduce

import cufflinks as cf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV,cross_val_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer

sns.set()
cf.go_offline()
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

In [2]:
ruta='/home/asm/amv/credit_scoring/datos/GiveMeSomeCredit'

In [3]:
df=pd.read_csv(ruta+'/cs-training.csv',index_col=0)
df.reset_index(drop=True,inplace=True)
df.insert(0,'ID',df.index+1)
df.set_index('ID',inplace=True)
df.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
ID,,,,,,,,,,,
1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [4]:
target=['SeriousDlqin2yrs']

varc=['RevolvingUtilizationOfUnsecuredLines','DebtRatio',
      'MonthlyIncome']

vard=['NumberOfTime30-59DaysPastDueNotWorse','NumberOfTimes90DaysLate',
      'NumberRealEstateLoansOrLines','NumberOfTime60-89DaysPastDueNotWorse','NumberOfDependents',
      'age','NumberOfOpenCreditLinesAndLoans']

In [5]:
assert len(varc)+len(vard)+len(target)==len(df.columns)

In [6]:
train,test=train_test_split(df,test_size=0.2,random_state=42,stratify=df[target]) 
train.shape,test.shape   

((120000, 11), (30000, 11))

In [7]:
train[varc+vard].describe(percentiles=[.01,.05,.1,.25,.5,.75,.9,.95,.99]).round(2)

,RevolvingUtilizationOfUnsecuredLines,DebtRatio,MonthlyIncome,NumberOfTime30-59DaysPastDueNotWorse,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,age,NumberOfOpenCreditLinesAndLoans
count,120000.00,120000.00,96325.00,120000.00,120000.00,120000.00,120000.00,116872.00,120000.00,120000.00
mean,6.13,352.27,6651.51,0.42,0.26,1.02,0.24,0.76,52.29,8.47
std,253.36,2093.71,14541.18,4.18,4.16,1.13,4.14,1.12,14.77,5.16
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,24.00,0.00
5%,0.00,0.00,1280.00,0.00,0.00,0.00,0.00,0.00,29.00,2.00
10%,0.00,0.03,2000.00,0.00,0.00,0.00,0.00,0.00,33.00,3.00
25%,0.03,0.18,3400.00,0.00,0.00,0.00,0.00,0.00,41.00,5.00
50%,0.15,0.37,5390.00,0.00,0.00,1.00,0.00,0.00,52.00,8.00
75%,0.56,0.86,8238.00,0.00,0.00,2.00,0.00,1.00,63.00,11.00


In [8]:
def outliers(df:pd.DataFrame,lower:float=.01,upper:float=.99)->tuple:
    X=df.copy()
    l,u=X.quantile([lower]).values,X.quantile([upper]).values
    X['extremo']=np.where((X>u).sum(axis=1)+(X<l).sum(axis=1)==0,0,1)
    no_outliers=X[X['extremo']==0].drop('extremo',axis=1)
    return no_outliers

In [9]:
train_=outliers(train)
train_.describe(percentiles=[.01,.05,.10,.25,.50,.75,.90,.95,.99]).round(2)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,111037.00,111037.00,111037.00,111037.00,111037.00,89785.00,111037.00,111037.00,111037.00,111037.00,108286.00
mean,0.06,0.31,52.33,0.21,277.39,6151.57,8.29,0.06,0.96,0.05,0.73
std,0.23,0.34,14.32,0.58,789.82,3903.07,4.69,0.31,0.93,0.24,1.05
min,0.00,0.00,24.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1%,0.00,0.00,25.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5%,0.00,0.00,29.00,0.00,0.01,1362.60,2.00,0.00,0.00,0.00,0.00
10%,0.00,0.00,33.00,0.00,0.04,2083.00,3.00,0.00,0.00,0.00,0.00
25%,0.00,0.03,41.00,0.00,0.18,3400.00,5.00,0.00,0.00,0.00,0.00
50%,0.00,0.14,52.00,0.00,0.36,5333.00,8.00,0.00,1.00,0.00,0.00
75%,0.00,0.53,63.00,0.00,0.81,8064.00,11.00,0.00,2.00,0.00,1.00


In [10]:
imp_c=SimpleImputer(strategy='mean')
imp_d=SimpleImputer(strategy='most_frequent')
imp_c.fit(train_[varc]),imp_d.fit(train_[vard])

(SimpleImputer(), SimpleImputer(strategy='most_frequent'))

In [11]:
train_[varc]=imp_c.transform(train_[varc])
train_[vard]=imp_d.transform(train_[vard])
train_.describe(percentiles=[.01,.05,.10,.25,.50,.75,.90,.95,.99]).round(2)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,111037.00,111037.00,111037.00,111037.00,111037.00,111037.00,111037.00,111037.00,111037.00,111037.00,111037.00
mean,0.06,0.31,52.33,0.21,277.39,6151.57,8.29,0.06,0.96,0.05,0.71
std,0.23,0.34,14.32,0.58,789.82,3509.73,4.69,0.31,0.93,0.24,1.04
min,0.00,0.00,24.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1%,0.00,0.00,25.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
5%,0.00,0.00,29.00,0.00,0.01,1567.00,2.00,0.00,0.00,0.00,0.00
10%,0.00,0.00,33.00,0.00,0.04,2355.00,3.00,0.00,0.00,0.00,0.00
25%,0.00,0.03,41.00,0.00,0.18,3895.00,5.00,0.00,0.00,0.00,0.00
50%,0.00,0.14,52.00,0.00,0.36,6151.57,8.00,0.00,1.00,0.00,0.00
75%,0.00,0.53,63.00,0.00,0.81,7291.00,11.00,0.00,2.00,0.00,1.00


In [12]:
from xgboost import XGBClassifier

In [13]:
xgb=XGBClassifier()
grid={'n_estimators':range(10,501,10),'max_depth':range(1,4),'learning_rate':[.0001,.001,.01,.1,1],'n_jobs':[-1],
      'subsample':[.1,.2,.3,.4,.5,.6,.7,.8,.9,1],'random_state':[0]}
xgb_grid=RandomizedSearchCV(xgb,grid,cv=10,scoring='roc_auc',n_iter=50,n_jobs=-1,random_state=123)
xgb_grid.fit(train_[varc+vard],train_[target])
model=xgb_grid.best_estimator_
model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=170, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=0, ...)

In [14]:
roc_auc_score(train_[target],model.predict_proba(train_[varc+vard])[:,1])

0.8612700183349418

In [15]:
test_imp = test.copy()
test_imp[varc]=imp_c.transform(test[varc])
test_imp[vard]=imp_d.transform(test[vard])
test_imp.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
ID,,,,,,,,,,,
10683,0,0.019252,66.0,0.0,0.433926,5175.0,13.0,0.0,1.0,0.0,0.0
40174,0,0.454221,78.0,0.0,0.873533,2300.0,7.0,0.0,1.0,0.0,2.0
4720,0,0.077572,59.0,0.0,0.172882,10833.0,5.0,0.0,1.0,0.0,0.0
81731,0,0.107864,59.0,0.0,0.453692,9900.0,10.0,0.0,2.0,0.0,0.0
113640,0,0.101095,67.0,0.0,0.407831,5950.0,14.0,0.0,2.0,0.0,0.0


In [16]:
roc_auc_score(test_imp[target],model.predict_proba(test_imp[varc+vard])[:,1])

0.8664393490287498

In [17]:
pred=pd.read_csv(ruta+'/cs-test.csv')
pred['Id']=pred['Unnamed: 0']
pred[varc]=imp_c.transform(pred[varc])
pred[vard]=imp_d.transform(pred[vard])
pred['Probability']=model.predict_proba(pred[varc+vard])[:,1]
pred

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,Id,Probability
0,1,NaN,0.885519,43.0,0.0,0.177513,5700.000000,4.0,0.0,0.0,0.0,0.0,1,0.067352
1,2,NaN,0.463295,57.0,0.0,0.527237,9141.000000,15.0,0.0,4.0,0.0,2.0,2,0.041734
2,3,NaN,0.043275,59.0,0.0,0.687648,5083.000000,12.0,0.0,1.0,0.0,2.0,3,0.014652
3,4,NaN,0.280308,38.0,1.0,0.925961,3200.000000,7.0,0.0,2.0,0.0,0.0,4,0.081005
4,5,NaN,1.000000,27.0,0.0,0.019917,3865.000000,4.0,0.0,0.0,0.0,1.0,5,0.086479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101498,101499,NaN,0.282653,24.0,0.0,0.068522,1400.000000,5.0,0.0,0.0,0.0,0.0,101499,0.039344
101499,101500,NaN,0.922156,36.0,3.0,0.934217,7615.000000,8.0,0.0,2.0,0.0,4.0,101500,0.347970
101500,101501,NaN,0.081596,70.0,0.0,836.000000,6151.573915,3.0,0.0,0.0,0.0,0.0,101501,0.007596
101501,101502,NaN,0.335457,56.0,0.0,3568.000000,6151.573915,8.0,0.0,2.0,1.0,3.0,101502,0.077450


In [18]:
pred[['Id','Probability']].to_csv('xgboost.csv', index=False)